# Base de dados Caltech 256

- Base de dados: https://www.kaggle.com/datasets/jessicali9530/caltech256
- Código baseado em: https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/imageclassification_caltech/Image-classification-fulltraining-highlevel.html e https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/imageclassification_caltech/Image-classification-fulltraining.html

In [10]:
import urllib.request
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import numpy as np
import os

In [11]:
def download(url):
    arquivo = url.split('/')[-1]
    #print(arquivo)
    if not os.path.exists(arquivo):
        urllib.request.urlretrieve(url, arquivo)

In [12]:
download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')

In [13]:
download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')

In [14]:
session = sagemaker.Session()
role = get_execution_role()
bucket = 'redeneural'
subpasta_dataset = 'caltech'
s3_treinamento = 's3://{}/{}/train/'.format(bucket, subpasta_dataset)
s3_teste = 's3://{}/{}/validation/'.format(bucket, subpasta_dataset)
s3_output_location = 's3://{}/{}/output'.format(bucket, subpasta_dataset)
print('Localização da base de treinamento: ', s3_treinamento)
print('Localização da base de teste: ', s3_teste)
print('Localização do modelo final: ', s3_output_location)

Localização da base de treinamento:  s3://redeneural/caltech/train/
Localização da base de teste:  s3://redeneural/caltech/validation/
Localização do modelo final:  s3://redeneural/caltech/output


In [16]:
!aws s3 cp caltech-256-60-train.rec $s3_treinamento --quiet
!aws s3 cp caltech-256-60-val.rec $s3_teste --quiet

# Configurações e treinamento da rede neural

In [17]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
container = sagemaker.image_uris.retrieve(framework = 'image-classification', 
                                          region = boto3.Session().region_name)

In [19]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
rede_neural = sagemaker.estimator.Estimator(image_uri = container,
                                            role = role,
                                            instance_count = 1,
                                            instance_type = 'ml.p3.2xlarge',
                                            input_mode = 'File',
                                            output_path = s3_output_location) # spot instances

In [21]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html
# Resnet: https://arxiv.org/pdf/1512.03385.pdf
rede_neural.set_hyperparameters(num_layers = 18,
                                image_shape = '3,224,224',
                                num_classes = 257,
                                num_training_samples = 15420,
                                mini_batch_size = 128,
                                epochs = 5,
                                learning_rate = 0.01)

In [22]:
train_data = sagemaker.inputs.TrainingInput(s3_treinamento,
                                            content_type = 'application/x-recordio',
                                            s3_data_type = 'S3Prefix')
test_data = sagemaker.inputs.TrainingInput(s3_teste,
                                            content_type = 'application/x-recordio',
                                            s3_data_type = 'S3Prefix')

In [24]:
data_channels = {'train': train_data, 'validation': test_data}
rede_neural.fit(inputs = data_channels, logs = True)

2022-05-31 14:27:20 Starting - Starting the training job...
2022-05-31 14:27:49 Starting - Preparing the instances for trainingProfilerReport-1654007240: InProgress
.........
2022-05-31 14:29:10 Downloading - Downloading input data...
2022-05-31 14:29:47 Training - Downloading the training image........Docker entrypoint called with argument(s): train
[05/31/2022 14:31:09 INFO 139664079087424] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[05/31/2022 14:31:09 INFO 139664079087424] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {'epochs': '5', 'image_shape': '3,224,224', 'learn

# Deploy e previsões

In [25]:
rede_neural_classificador = rede_neural.deploy(initial_instance_count = 1, 
                                               instance_type = 'ml.m4.xlarge')

-------!

In [26]:
imagem = open('test.jpg', 'rb').read()

In [29]:
runtime = boto3.Session().client(service_name = 'sagemaker-runtime')
resposta = runtime.invoke_endpoint(EndpointName = 'image-classification-2022-05-31-14-41-13-204',
                                   ContentType = 'application/x-image',
                                   Body = bytearray(imagem))

In [30]:
resposta

{'ResponseMetadata': {'RequestId': 'b5fbfcad-43d2-4f59-9113-4217d16611ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b5fbfcad-43d2-4f59-9113-4217d16611ef',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Tue, 31 May 2022 14:49:28 GMT',
   'content-type': 'application/json',
   'content-length': '5808'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f308087a8d0>}

In [31]:
resultado = resposta['Body'].read()
resultado = json.loads(resultado)
resultado

[0.0019974715542048216,
 0.0002946134191006422,
 0.00028446977376006544,
 0.006339970976114273,
 0.0011686066864058375,
 0.0010097363265231252,
 0.001309880055487156,
 0.01974247395992279,
 0.008100583218038082,
 0.003078090725466609,
 0.0006036317790858448,
 0.00015049017383717,
 0.003128022886812687,
 0.0005472539342008531,
 0.0009612985304556787,
 5.4310909035848454e-05,
 0.0017858338542282581,
 0.00262169586494565,
 0.00031083906651474535,
 0.011439608410000801,
 0.001308924751356244,
 0.000738618487957865,
 0.0006177397444844246,
 0.0005302262725308537,
 0.0004987695720046759,
 0.004640379920601845,
 0.00048101108404807746,
 0.002151242457330227,
 0.00022024611826054752,
 0.0035133911296725273,
 0.00027948542265221477,
 0.00359237939119339,
 0.0008572120568715036,
 0.00336466240696609,
 0.00015572071424685419,
 0.0011712850537151098,
 0.00023849365243222564,
 0.0005042092525400221,
 0.00279337540268898,
 0.006062807980924845,
 0.002882768865674734,
 0.022409098222851753,
 0.002615

In [32]:
sum(resultado)

0.9999998895691533

In [33]:
len(resultado)

257

In [34]:
probabilidade_maior = np.argmax(resultado)
probabilidade_maior

136

In [35]:
categorias = ["ak47", "american-flag", "backpack", "baseball-bat", "baseball-glove",
    "basketball-hoop", "bat", "bathtub", "bear", "beer-mug", "billiards", "binoculars",
    "birdbath", "blimp", "bonsai-101", "boom-box", "bowling-ball", "bowling-pin",
    "boxing-glove", "brain-101", "breadmaker", "buddha-101", "bulldozer", "butterfly",
    "cactus", "cake", "calculator", "camel", "cannon", "canoe", "car-tire",
    "cartman", "cd", "centipede", "cereal-box", "chandelier-101", "chess-board",
    "chimp", "chopsticks", "cockroach", "coffee-mug", "coffin", "coin", "comet",
    "computer-keyboard", "computer-monitor", "computer-mouse", "conch", "cormorant",
    "covered-wagon", "cowboy-hat", "crab-101", "desk-globe", "diamond-ring",
    "dice", "dog", "dolphin-101", "doorknob", "drinking-straw", "duck", "dumb-bell",
    "eiffel-tower", "electric-guitar-101", "elephant-101", "elk", "ewer-101",
    "eyeglasses", "fern", "fighter-jet", "fire-extinguisher", "fire-hydrant",
    "fire-truck", "fireworks", "flashlight", "floppy-disk", "football-helmet",
    "french-horn", "fried-egg", "frisbee", "frog", "frying-pan", "galaxy",
    "gas-pump", "giraffe", "goat", "golden-gate-bridge", "goldfish", "golf-ball",
    "goose", "gorilla", "grand-piano-101", "grapes", "grasshopper", "guitar-pick",
    "hamburger", "hammock", "harmonica", "harp", "harpsichord", "hawksbill-101",
    "head-phones", "helicopter-101", "hibiscus", "homer-simpson", "horse",
    "horseshoe-crab", "hot-air-balloon", "hot-dog", "hot-tub", "hourglass",
    "house-fly", "human-skeleton", "hummingbird", "ibis-101", "ice-cream-cone",
    "iguana", "ipod", "iris", "jesus-christ", "joy-stick", "kangaroo-101",
    "kayak", "ketch-101", "killer-whale", "knife", "ladder", "laptop-101", "lathe",
    "leopards-101", "license-plate", "lightbulb", "light-house", "lightning",
    "llama-101", "mailbox", "mandolin", "mars", "mattress", "megaphone", "menorah-101",
    "microscope", "microwave", "minaret", "minotaur", "motorbikes-101", "mountain-bike",
    "mushroom", "mussels", "necktie", "octopus", "ostrich", "owl", "palm-pilot",
    "palm-tree", "paperclip", "paper-shredder", "pci-card", "penguin", "people",
    "pez-dispenser", "photocopier", "picnic-table", "playing-card", "porcupine",
    "pram", "praying-mantis", "pyramid", "raccoon", "radio-telescope", "rainbow",
    "refrigerator", "revolver-101", "rifle", "rotary-phone", "roulette-wheel",
    "saddle", "saturn", "school-bus", "scorpion-101", "screwdriver", "segway",
    "self-propelled-lawn-mower", "sextant", "sheet-music", "skateboard", "skunk",
    "skyscraper", "smokestack", "snail", "snake", "sneaker", "snowmobile", "soccer-ball",
    "socks", "soda-can", "spaghetti", "speed-boat", "spider", "spoon", "stained-glass",
    "starfish-101", "steering-wheel", "stirrups", "sunflower-101", "superman", "sushi",
    "swan", "swiss-army-knife", "sword", "syringe", "tambourine", "teapot", "teddy-bear",
    "teepee", "telephone-box", "tennis-ball", "tennis-court", "tennis-racket",
    "theodolite", "toaster", "tomato", "tombstone", "top-hat", "touring-bike",
    "tower-pisa", "traffic-light", "treadmill", "triceratops", "tricycle", "trilobite-101",
    "tripod", "t-shirt", "tuning-fork", "tweezer", "umbrella-101", "unicorn", "vcr",
    "video-projector", "washing-machine", "watch-101", "waterfall", "watermelon", "welding-mask",
    "wheelbarrow", "windmill", "wine-bottle", "xylophone", "yarmulke", "yo-yo", "zebra",
    "airplanes-101", "car-side-101", "faces-easy-101", "greyhound", "tennis-shoes",
    "toad", "clutter"]

In [36]:
rotulo = categorias[probabilidade_maior]
rotulo

'mars'

In [37]:
rede_neural_classificador.delete_endpoint()